Bibliothèques utiles :

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from cfd_common import *
from math import *

In [2]:
%matplotlib qt

In [3]:
plt.ion()

Choix du domaine et des paramètres de la discrétisation du problème :

In [4]:
##### Domain #####

# Domain size
Lx = 2.
Ly = 2.

# Physical grid size
Nx_phys = 200
Ny_phys = 200
Δx = Lx/Nx_phys
Δy = Ly/Ny_phys

# Numerical grid size, including ghost points
Nx = Nx_phys + 2
Ny = Ny_phys + 2

# Physical grid (x,y)
# Due to the "staggered-like" approach to boundary conditions with ghost points,
#  the border go through the middle of border cells
x = np.linspace(Δx/2, Lx-Δx/2, Nx_phys) 
y = np.linspace(Δy/2, Ly-Δy/2, Ny_phys)
[xx,yy] = np.meshgrid(x,y) 

Initialisation des champs de vitesse / champ de pression et champ de densité :

On note $u,v$ les composantes horizontale, verticales du champ de vitesse. On les initialise à :...

In [5]:
##### Fields #####

# Velocity field (u,v)[i,j], initialized to "zero"
u = np.zeros((Nx,Ny))
v = np.zeros((Nx,Ny))

vrand_ampli = 0.1
v += vrand_ampli * ( 1-2*np.random.rand(Nx,Ny) ) # random vertical velocity to speed up the instability

# Pressure field press[i,j] and its gradient (dx_press,dy_press)
press = np.zeros((Nx,Ny))
dx_press = np.zeros((Nx,Ny))
dy_press = np.zeros((Nx,Ny))

# Density field
ρ0 = 5
ρ = ρ0*np.ones((Nx,Ny))
#ρ[:, 4*Ny//10 : 8*Ny//10] = 1
#ρ[1:-1,1:-1] = np.where(((xx-0*Lx/2)**2+(yy-Ly/2)**2)<0.5, 5, 1)
#ρ[1:-1,1:-1] = np.where(((xx-0*Lx/2)**2+(yy-Ly/2)**2)<0.3, 1, ρ[1:-1,1:-1])
ρ[1:-1,1:-1] = np.transpose(np.where(((xx-Lx/2)**2+(yy-0.15*Ly)**2)<0.2**2, 1, 5))

On impose les conditions aux bords (sur les GhostPoints) :

In [6]:
def VelocityGhostPoints(u,v):
    # ya pas cette histoire de "frontière = au milieu des cellules de bord ?"
    ### left
    u[0, :] = 0#u[1, :]
    v[0, :] = 0#v[1, :]
    ### right    
    u[-1, :] = 0#u[-2, :]    # outflow condition => dérivée nulle
    v[-1, :] = 0#v[-2, :]    # outflow condition => dérivée nulle
    ### bottom
    u[:,  0] = 0  # no slip
    v[:,  0] = 0  # imperméabilité
    ### top     
    u[:, -1] = u[:, -2]
    v[:, -1] = v[:, -2]
    
def PhiGhostPoints(phi):
    ### left
    phi[0,  1:-1] = 2 * (-0.0001*ρ0*g*y) - phi[1,  1:-1]
    phi[0, :] = phi[1, :]
    ### right
    phi[-1, 1:-1] = 2 * (-0.0001*ρ0*g*y) - phi[-2, 1:-1]
    phi[-1, :] = phi[-2, :]
    ### bottom               
    phi[:,  0] = phi[:,  1]
    ### top            
    phi[:, -1] = - phi[:, -2]

def DensityGhostPoints(ρ):
    # ghost points for density : Neumann ∂ρ=0 boundary condition
    ρ[ 0, :] = ρ0 #ρ[1, :] # left
    ρ[-1, :] = ρ0 #ρ[-2, :] # right
    ρ[:,  0] = ρ0 #ρ[:, 1] # bottom
    ρ[:, -1] = ρ0 #ρ[:, -2] # top  

Ensuite vient la définition de notre schéma :

In [7]:
###### CONSTRUCTION des matrices et LU decomposition

### Matrix construction for projection step
LAPN = FD_2D_Laplacian_matrix (Nx_phys, Ny_phys, Δx, Δy, BCdir_left=False, BCdir_right=False, BCdir_top=True, BCdir_bot=False) 
LUPN = LUdecomposition(LAPN)

In [8]:
## Check mass
def mass_check(rho):
    return Δx*Δy*np.sum(rho)

mass_check(ρ)

19.8964

Puis la boucle principale de calcul :

In [ ]:
### ATTENTION: dt_init calculer la CFL a chaque iteration... 
Δt = 0.000005

t = 0. # total time

# parameters
Re = 10000 # Reynolds number
g = 1
α = 0*pi/4

# how much do we correct for diffusion in semi-lag advection at the price of dispersion
ρ_semilag2_coeff = 0.8
u_semilag2_coeff = 0.7

fig = plt.figure(1, figsize=(14,7))

loop_continue = True
def on_close (event):
    global loop_continue
    loop_continue = False
fig.canvas.mpl_connect('close_event', on_close)


niter = 0
disp_modulo = 1000

In [10]:
while loop_continue:

    ###### Advection semi-lagrangienne
    adv_u = SemiLag2(u,v, u, Δx,Δy,Δt, u_semilag2_coeff)
    adv_v = SemiLag2(u,v, v, Δx,Δy,Δt, u_semilag2_coeff)

    if ρ_semilag2_coeff is None:
        ρ = SemiLag(u,v, ρ, Δx,Δy,Δt)
    else:
        ρ = SemiLag2(u,v, ρ, Δx,Δy,Δt, ρ_semilag2_coeff)
    
    ρ = np.maximum(ρ, 0.2)
    
    ### Mise à jour des conditions limites pour la densité.
    DensityGhostPoints(ρ)
    
    ###### Diffusion step
    ustar = adv_u + Δt*Laplacian(u, Δx,Δy)/Re/ρ
    vstar = adv_v + Δt*Laplacian(v, Δx,Δy)/Re/ρ
    
    ustar += +sin(α) * g * Δt
    vstar += -cos(α) * g * Δt
    
    ###### Mise à jour des conditions limites pour la vitesse :
    VelocityGhostPoints(ustar,vstar)
 
    ### Update divstar 
    divstar = Divergence(ustar,vstar, Δx,Δy)
#    divstar = divstar - np.mean(divstar[1:-1,1:-1])
    
    ### Solving the linear system
    press[1:-1,1:-1] = Resolve(LUPN, RHS=divstar[1:-1,1:-1])/Δt/10000

    ### Mise à jour des conditions limites pour la pression :
    PhiGhostPoints(press)

    ### Update gradphi
    dx_press[1:-1, :] = (press[2:, :] - press[:-2, :])/Δx/2
    dy_press[:, 1:-1] = (press[:, 2:] - press[:, :-2])/Δy/2

    ### Project u
    u = ustar - Δt*10000*dx_press / ρ
    v = vstar - Δt*10000*dy_press / ρ
    
    VelocityGhostPoints(u,v)
    

    if (niter%disp_modulo == 0):
        #print(mass_check(ρ))
        fig.clear()
        (ax1,ax2) = fig.subplots(nrows=1, ncols=2, sharey=True)
        
        def plot_vel (ax, modx=None, mody=None, vmax=20):
            if modx is None:
                modx = Nx_phys//50
            if mody is None:
                mody = Ny_phys//50
            ax.quiver(xx[::modx,::mody], yy[::modx,::mody], np.transpose(u[1:-1:modx,1:-1:mody]), np.transpose(v[1:-1:modx,1:-1:mody]), scale=vmax*2)
            ax.set_aspect('equal', adjustable='box')
    
        fig.suptitle(r"Velocity, Density and Pressure fields at $t={:.3f}$. No-slip B.C., free-flow B.C. top, $Re={}$, SemiLag2({}) for $\vec{{u}}$, ${}\times{}$ grid.{} SemiLag2({}) for $\rho$ (min 0.2), total mass : {:.2f}".format(t,Re,u_semilag2_coeff,Nx_phys,Ny_phys,"\n",ρ_semilag2_coeff,mass_check(ρ)))
        
        ax1.clear()
        ax1.imshow(np.transpose(ρ[1:-1,1:-1]), origin='lower', extent=(0,Lx,0,Ly), cmap=plt.cm.RdPu, vmin=0, vmax=6)
        plot_vel(ax1)
        
        ax2.clear()
        im = ax2.imshow(np.transpose(1e4*press[1:-1,1:-1]), origin='lower', extent=(0,Lx,0,Ly), cmap='plasma', vmax=ρ0*g*Ly, vmin=0)#, vmax=ρ0*1, vmin=-ρ0*2.5)
        fig.colorbar(im)
        plot_vel(ax2)
        
#        my_cmap = plt.cm.gnuplot2_r(np.arange(plt.cm.gnuplot2_r.N))
#        my_cmap = 0.5 * (1+my_cmap)
#        my_cmap = matplotlib.colors.ListedColormap(my_cmap)
#        ax1.imshow(np.transpose(np.sqrt(u[1:-1,1:-1]**2 + v[1:-1,1:-1]**2)), origin='lower', extent=(0,Lx,0,Ly), cmap=my_cmap, vmin=0, vmax=2)
#        plot_vel(ax1)
    

        fig.tight_layout()
        if True and niter != 0:
            plt.savefig('bubble6/{:04d}.png'.format(niter//disp_modulo))
        plt.draw()
        
        mpl_pause_background(0.00001)
        
    t += Δt
    niter += 1